In [3]:
import requests
import re
from collections import defaultdict
from unidecode import unidecode
from itertools import chain
from bs4 import BeautifulSoup

In [7]:
class BillFinder:
    
    def __init__(self):
        
        self.source_urls_energy = {'Canada': 'https://en.wikipedia.org/wiki/List_of_Canadian_electric_utilities',
                                   'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_electric_companies',
                                   'UK': 'https://www.utilitysavingexpert.com/energy/suppliers/',
                                   'Australia': 'https://www.canstarblue.com.au/electricity/largest-energy-companies-australia/',
                                   'New Zealand': 'https://en.wikipedia.org/wiki/New_Zealand_electricity_market'}
        
        self.source_urls_water = {'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_water_companies',
                                  'UK': 'https://en.wikipedia.org/wiki/United_Kingdom_water_companies'}
        
        self.source_urls_gas = {'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_natural_gas_companies',
                                'UK': 'https://en.wikipedia.org/wiki/List_of_British_natural_gas_companies'}
        
        self.stop_words = 'company inc corp corporation ltd authority llc district department commission cooperative association limited'.split()
        
        self.collected_companies = defaultdict(list)
    
    def get_energy_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_energy[country]).text)
        
        if country == 'Canada':
            for t in soup.find_all('table', role="presentation"):
                for _ in t.find_all('li'):
                    for a in _.find_all('a'):
                        company_set.add(a.text)
        elif country == 'USA':
            for t in soup.find('span', id="List_of_US_electric_companies_by_state"):
                for e in t.parent.next_elements:
                    if e.name == 'table':
                        break
                    if e.name == 'ul':
                        for _ in e.find_all('li'):
                            if company_name := _.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for header_text in ['List of UK energy suppliers', 'List of business energy suppliers']:
                p = soup.find('h2', string=header_text).parent
                for _ in p.find_all('div', class_="table-responsive"):
                    for row in _.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[1].text)
                            
        elif country == 'New Zealand':
            for _ in soup.find('caption').next_siblings:
                if _.name == 'tbody':
                    for row in _.find_all('tr'):
                        if c := row.find('td'):
                            company_set.add(c.text)
                    break
        elif country == 'Australia':
            for s in soup.find('h2', id='biggest').next_siblings:
                if s.name == 'ul':
                    for item_ in s.find_all('li'):
                        company_set.add(item_.find('strong').text)
                    break
            
                            
        print(f'{country}: found {len(company_set):,} energy companies')
        
        self.collected_companies['energy'].extend(sorted(company_set))
                        
        return self
    
    def get_water_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_water[country]).text)
        
        if country == 'USA':
            for t in soup.find_all('h2'):
                if t.find_all('a', href=True):
                    if list_of_company_names := t.find_next_sibling('ul'):
                        for name_line in list_of_company_names.find_all('li'):
                            if company_name := name_line.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for s in soup.find('span', id='Water_and_sewerage').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
            for s in soup.find('span', id='Water_only').parent.next_siblings:
                if s.name == 'table':
                    for row in s.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[0].text.strip()) 
                    break
                    
            for id_ in ['Scotland', 'Northern_Ireland', 'Crown_dependencies']:
                for s in soup.find('span', id=id_).parent.next_siblings:
                    if s.name == 'ul':
                        for item_ in s.find_all('li'):
                            company_set.add(item_.text)
                        break
                            
        print(f'{country}: found {len(company_set):,} water companies')
                        
        self.collected_companies['water'].extend(sorted(company_set))
                        
        return self
    
    def get_gas_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls_gas[country]).text)
        
        if country == 'USA':
            for list_item in soup.find('span', id='List').parent.find_next_sibling('ul').find_all('li'):
                for c in unidecode(list_item.text).split('-')[1].split(','):
                    if company_name := c.strip():
                        company_set.add(company_name)
        elif country == 'UK':
            for _ in soup.find(lambda t: (t.name == 'p') and ('Utilities' in t.text)).next_siblings:
                if _.name == 'ul':
                    for list_item in _.find_all('li'):
                        if s := list_item.text.strip():
                            if '(' not in s:
                                company_set.add(s)
                            else:
                                for st in re.split(r'[(),]| and ', s):
                                    if potential_name := ' '.join([w for w in st.split() if (w.isupper() or w.istitle())]).strip():
                                        company_set.add(potential_name)
                    break

        print(f'{country}: found {len(company_set):,} gas companies')
        
        self.collected_companies['gas'].extend(sorted(company_set))
                        
        return self
    
    def normalise(self, name):
        
        okayed_name_parts = []
        
        for _ in re.sub(r"[,.;:']", '', name).lower().split():
            if w := _.strip():
                if (len(w) > 2) and (w not in self.stop_words):
                    okayed_name_parts.append(w)
        
        s = ' '.join(okayed_name_parts).split('[')[0]
        
        if ('(via ' not in s) and ('(' in s):
            s = s.split('(')[0]
            
        return s.strip()
    
    def review_company_names(self):
        
        for company_type in self.collected_companies:
            self.collected_companies[company_type] = sorted({normalised for name in self.collected_companies[company_type] if (normalised := self.normalise(name))})
            
        print(f'collected {sum({len(self.collected_companies[company_type]) for company_type in self.collected_companies}):,} unique companies')

        return self

In [8]:
bf = BillFinder()

In [9]:
bf.get_gas_companies(country='UK') \
        .get_gas_companies(country='USA') \
        .get_water_companies('USA') \
        .get_water_companies('UK') \
        .get_energy_companies('Australia') \
        .get_energy_companies('UK') \
        .get_energy_companies('Canada') \
        .review_company_names()

UK: found 23 gas companies
USA: found 159 gas companies
USA: found 558 water companies
UK: found 30 water companies
Australia: found 23 energy companies
UK: found 75 energy companies
Canada: found 96 energy companies
collected 951 unique companies


In [10]:
bf.collected_companies['energy']

['1st energy',
 'actewagl',
 'advanti gas',
 'affect energy',
 'agl',
 'alectra utilities',
 'algoma power',
 'alinta energy',
 'altalink',
 'angelic energy',
 'antigonish electric utility',
 'atco electric',
 'atco electric yukon',
 'aurora energy',
 'avanti gas',
 'avro energy',
 'battle river power',
 'beam energy',
 'beaver rea',
 'berwick electric light',
 'better energy',
 'blue mountain power',
 'boost power',
 'boralex',
 'borradaile rea',
 'braes rea',
 'bristol energy',
 'british gas',
 'british gas business',
 'bulb energy',
 'canadian niagara power',
 'canso electric light',
 'cardiff energy supply',
 'cartier wind energy',
 'churchill falls',
 'city lethbridge electric utility',
 'city medicine hat electric utility',
 'city red deer electric light and power',
 'claysmore rea',
 'clear business energy',
 'co-operative energy',
 'columbia power',
 'contract natrual gas',
 'cornwall electric',
 'corona energy',
 'covau',
 'crown gas power',
 'daligas',
 'devonia rea',
 'diamo